In [128]:
import pandas as pd
import re
from konlpy.tag import Okt
# 텍스트 데이터를 머신러닝 알고리즘이 처리할 수 있는 수치 벡터로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import re
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("C:/Users/82105/Downloads/divorce_df.csv", encoding="utf-8-sig")

In [3]:
print("=== 데이터 전처리: null 값 제거 ===")
print(f"전처리 전 데이터 수: {len(df)}")

# null 값이 있는 행 제거
df_clean = df.dropna(subset=['input']).copy()
print(f"null 값 제거 후 데이터 수: {len(df_clean)}")
print()

=== 데이터 전처리: null 값 제거 ===
전처리 전 데이터 수: 741
null 값 제거 후 데이터 수: 711



# 1단계 : KoNLPy를 사용한 정교한 전처리

In [45]:
MINIMAL_STOPWORDS = [
    '것', '수', '때', '등', '들', '더', '이', '그', '저', '나', 
    '우리', '같', '또', '만', '년', '월', '일', '하다', '있다', '되다', '이르다',
    '이다', '들다', '되어다', '다른', '따르다'
     '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다',
    '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해',
    '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자','제호','제조',

     '없', '있', '하', '되', '않', 
    '나', '우리', '너', '당신', '같', '또', '것', '때', '등', 
    '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과', '사람',

    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지''하나요', '위해', '이혼',
     '조건', '대한', '관련', '따르다', '판단', '인정', '적용', 
    '효력', '성립', '발생', '요건', '증명', '근거', '주장','이혼' , '가지', '가지다', '가하다', '이다', '들다', '되다', '따라서', '거나'
]
# 중복 제거 후 사용
MINIMAL_STOPWORDS = list(set(MINIMAL_STOPWORDS))

# # 법률 전문용어는 무조건 포함
LEGAL_KEYWORDS = [ '위자료', '재산분할', '양육권', '친권', '면접교섭', 
    '협의이혼', '청구', '배상', '손해', '책임',
    '차용금', '반환', '취소', '원상회복', '사해행위', '채권자', '배우자', '이혼사유',
    '사유', 
    '혼인', '금전거래', '청구권',  '액수', '정해지', '부적법',
    '혼인파탄', '파탄', '분할', '양육비', '면접',
    '교섭', '협의', '조정신청', '손해배상', '부부', '배우자',
]

In [42]:
def preprocess_text(text):  # 전처리 함수 
    """
    한국어 텍스트를 입력받아 전처리하는 함수:
    1. '제3자' 형태의 단어를 임시 토큰으로 보호/복원하여 숫자를 보존.
    2. 불필요한 한글 이외의 문자를 제거.
    3. 형태소 분석 및 어간 추출.
    4. 불용어 및 1글자 단어 제거.
    """
    # 1단계: 한글, 공백을 제외한 모든 문자 제거
    # re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)는 한글과 공백만 남기고 나머지는 지우라는 의미
    # 제숫자 + 한글 글자 + 선택적 조사까지 포함
    # 딕셔너리를 함수 내부에 선언하여 매 호출마다 초기화 (핵심 수정 사항)
    protected_matches = {}

    def protect_term(match):
        # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
        token = f"###TOKEN{len(protected_matches)}###"
        protected_matches[token] = match.group(0)
        return match.group(0) # 일단 원본 단어 그대로 둔 채로 형태소 분석 진행
    def strip_josa(text):
    # 한글 명사 뒤 “의”, “가”, “을” 등 제거
        return re.sub(r'([가-힣]+)(의|가|를|은|는|과|와)$', r'\1', text)
    

    # 1단계: '제3자' 형태의 단어를 임시 토큰으로 치환하여 보호
    re.sub(r'(제\d+[가-힣]+)', protect_term, text)

    # 2단계: 한글, 공백, 보호 토큰의 문자(#)만 남기고 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣# ]', '', text)

    # 2단계: Okt 형태소 분석기를 이용한 토큰화 및 어간 추출(Stemming)
    # okt.pos(text, stem=True)는 문장을 (단어, 품사) 형태로 나누고, '하다', '먹다'처럼 원형으로 만들어줍니다.
    # 예: "먹었었고" -> ('먹다', 'Verb')
    # Okt 형태소 분석기 객체 생성
    okt = Okt() 
    word_tokens = okt.pos(text, stem=True)

    # 3단계: 불용어 제거
    # 형태소 분석 후 의미 있는 단어만 추출하는 필터링 과정
    # 의미를 가지는 명사, 동사, 형용사, 부사 중에서 1글자 이상인 단어만 추출합니다.
    # 품사 태그가 'Josa'(조사), 'Eomi'(어미), 'Punctuation'(구두점) 등인 단어들을 제거합니다.
    meaningful_words = []
    for word, pos in word_tokens:
        if word in LEGAL_KEYWORDS:
            meaningful_words.append(word)
        elif pos in ['Noun','Verb'] and len(word) > 1:
            token = strip_josa(word)
            if token not in MINIMAL_STOPWORDS:
                meaningful_words.append(token)
     # 6. 보호된 단어 강제 포함 및 복원 (핵심)
    # 보호 목록에 있던 단어들을 강제로 최종 리스트에 추가합니다.
    # (이미 리스트에 분해되어 들어갔을 수 있지만, 완벽한 복원을 위해 강제 추가)
    for original_term in protected_matches.values():
        # '제3자' 자체를 하나의 단어로 명시적으로 추가
        meaningful_words.append(original_term)
        
    # 7. 중복 제거 및 최종 문자열 반환
    final_words = list(dict.fromkeys(meaningful_words))
    # 최종적으로 공백으로 연결된 문자열을 반환합니다. 
    # 모델에 따라 리스트 형태(meaningful_words)를 그대로 사용할 수도 있습니다.
    # 텍스트에서 분석에 의미 있는 핵심 단어들만 남긴 리스트 생성
    return ' '.join(final_words)

In [48]:
# --- 4. 각 목적에 맞게 전처리 컬럼 생성 ---
# 질문 의도 파악 모델용: 'input' 컬럼만 전처리
# 'input' 컬럼에 전처리 함수를 적용하여 새로운 'input_processed' 컬럼 생성
df_clean['combined_processed'] = df_clean['text_combined'].apply(preprocess_text)

# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(df_clean[['text_combined', 'combined_processed']].head())

=== 각 목적에 맞게 생성된 전처리 컬럼들 ===
                                       text_combined  \
0  혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요...   
1  제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요? 제3자는...   
2  민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가...   
3  배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요? 위자료...   
4  사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...   

                                  combined_processed  
0  혼인 금전 거래 차용 반환 청구권 지급 입증 객관 금융 자료 또한 변제 독촉 증거 ...  
1  부부 일방 부정행위 법적 책임 지게 타인 생활 개입 파탄 초래 본질 방해 서다 침해...  
2  민법 규정 재판 사유 부정 행위 의미 간통 이르다 부부 정조 의무 일체 부정행위 포...  
3  배우자 부정행위 위자료 액수 정해지다 기간 결혼 가족 관계 부부 생활 영향 참작 장...  
4  사해행위 취소 원상회복 청구 소송 채권자 부적 법해 대해 진행 승소 판결 확정 재산...  


# 2단계: 최적화된 TF-IDF 벡터화

In [49]:
# 전처리된 텍스트를 다시 토큰 리스트 형태로 변환합니다.
texts = [doc.split() for doc in df_clean['combined_processed'].dropna()]

# Gensim의 Phrases 모델을 사용하여 통계적으로 의미 있는 bi-gram을 찾습니다.
# 이 모델은 '가하다 간주' 같은 우연한 조합은 무시하고, '정신적 고통' 같은 진짜 조합만 찾아냅니다.
phrases = Phrases(texts, min_count=5, threshold=10)
bigram_phraser = Phraser(phrases)

# bi-gram을 적용하여 토큰 리스트를 업데이트합니다. 예: ['정신적', '고통'] -> ['정신적_고통']
texts_with_ngrams = [bigram_phraser[doc] for doc in texts]


# --- 2. TF-IDF Vectorizer를 위한 'N-gram이 적용된' 텍스트 생성 ---
# TF-IDF는 문자열을 입력으로 받으므로, N-gram이 적용된 토큰들을 다시 공백으로 연결합니다.
df_clean.loc[df_clean['combined_processed'].notna(), 'ngram_text'] = [' '.join(doc) for doc in texts_with_ngrams]

In [50]:
# TF-IDF Vectorizer 설정
vectorizer = TfidfVectorizer(
    max_features=1000,      # 상위 빈도 1000개 단어만 사용
    min_df=7,           # 최소 등장 빈도를 조금 높여 노이즈를 더 줄일 수 있습니다.
    max_df=0.35,         # 최대 등장 비율을 조금 낮춰 너무 일반적인 단어를 제거합니다.           
    lowercase=False,         # 소문자 변환 x
    ngram_range=(1, 1),     # 1-gram, 2-gram 모두 사용
    sublinear_tf=True,            # TF 값에 로그 스케일 적용 (성능 향상)
    token_pattern=r'[\w_]+'     # '_'가 포함된 N-gram을 인식하도록 패턴 수정
)

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(df_clean['ngram_text'].dropna())  # X는 (문서 수 × 단어 수) 크기의 희소 행렬

print(f"TF-IDF 행렬 크기: {tfidf_matrix.shape}")
print(f"문서 수: {tfidf_matrix.shape[0]}")
print(f"특성(단어) 수: {tfidf_matrix.shape[1]}")

# 특성 이름(단어들) 확인
# vectorizer.get_feature_names_out()로 어떤 단어가 벡터에 들어갔는지 확인 가능
feature_names = vectorizer.get_feature_names_out()
print(f"상위 20개 특성: {feature_names[:20]}")

TF-IDF 행렬 크기: (711, 355)
문서 수: 711
특성(단어) 수: 355
상위 20개 특성: ['가능성' '가정법원' '가족' '가족_관계' '가해자' '각자' '간' '간주' '간통' '간통_포함' '개념' '개념_이르다'
 '개념_정조' '개별' '개입' '객관' '검토' '겪다' '결' '결정']


# 3단계: 토픽 개수 최적화
# 최적의 토픽 개수를 찾기 위해 여러 지표를 사용

# 더 정확한 토픽 모델링을 위해 Gensim 라이브러리를 사용
# 고급 기법: Gensim LDA + Coherence Score

In [51]:
from gensim.models.phrases import Phrases, Phraser

# 텍스트를 토큰 리스트로 변환
texts = [text.split() for text in df_clean['combined_processed']]

# 1. Bi-gram 모델 생성
phrases = Phrases(texts, min_count=5, threshold=10) # min_count, threshold는 데이터에 맞게 조정
bigram = Phraser(phrases)

# 2. Bi-gram 적용
texts_bigram = [bigram[doc] for doc in texts]

# (선택) Tri-gram까지 적용하고 싶다면 한 번 더 수행
# phrases_tri = Phrases(texts_bigram, min_count=5, threshold=10)
# trigram = Phraser(phrases_tri)
# texts_trigram = [trigram[doc] for doc in texts_bigram]

# 이제 texts_bigram (또는 texts_trigram)을 사용하여 Dictionary와 Corpus를 만드세요.
# dictionary = Dictionary(texts_bigram)
# corpus = [dictionary.doc2bow(text) for text in texts_bigram]

In [54]:
# Dictionary 생성
dictionary = Dictionary(texts_bigram)
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Corpus 생성  
corpus = [dictionary.doc2bow(text) for text in texts_bigram]

# 최적 토픽 수 찾기 (Coherence Score 사용)
coherence_scores = []
models_dict = {}
topic_range = range(2, 11)

for num_topics in topic_range:
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary, 
        num_topics=num_topics,
        random_state=42,
        passes=30,
        alpha='auto',        # 자동 조정
        per_word_topics=True
        
    )
    
    coherence_model = CoherenceModel(
        model=lda_model, 
        texts=texts_bigram, 
        dictionary=dictionary, 
        coherence='c_v'
    )
    
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    models_dict[num_topics] = lda_model
    print(f"토픽 수 {num_topics}: Coherence = {coherence_score:.4f}")

# 상위 3개 후보 선정
top_3_indices = np.argsort(coherence_scores)[-3:]
top_3_topics = [topic_range[i] for i in top_3_indices]
print(f"\nCoherence 상위 3개: {top_3_topics}")


# 각 후보의 실제 토픽 내용 확인
for num_topics in top_3_topics:
    print(f"\n=== {num_topics}개 토픽 결과 ===")
    model = models_dict[num_topics]
    
    for topic_id in range(num_topics):
        terms = model.show_topic(topic_id, topn=8)
        words = [term[0] for term in terms]
        print(f"토픽 {topic_id}: {', '.join(words)}")

토픽 수 2: Coherence = 0.4179
토픽 수 3: Coherence = 0.4006
토픽 수 4: Coherence = 0.4411
토픽 수 5: Coherence = 0.4070
토픽 수 6: Coherence = 0.4998
토픽 수 7: Coherence = 0.4458
토픽 수 8: Coherence = 0.4218
토픽 수 9: Coherence = 0.4485
토픽 수 10: Coherence = 0.4450

Coherence 상위 3개: [7, 9, 6]

=== 7개 토픽 결과 ===
토픽 0: 혼인, 부부, 실질, 생활, 파탄, 입증, 상태, 관계
토픽 1: 결정, 영향, 피해자, 종합_고려, 고려, 생활, 정신_고통, 부부
토픽 2: 불법행위, 정신_고통, 민법, 입증, 피해자, 부부, 입다, 손해배상_청구
토픽 3: 증거, 입증, 법원, 소송, 이르다, 법적, 행위, 상대방
토픽 4: 재산_분할, 민법, 청구, 이루어지다, 기여, 양육비, 기준, 포함
토픽 5: 정신_고통, 방해, 제3자가, 혼인_본질, 해당, 생활_유지, 침해, 부부
토픽 6: 정조_의무, 부부, 간통_포함, 행위, 평, 포함, 모든, 제3자가

=== 9개 토픽 결과 ===
토픽 0: 참작_평, 파탄_초래, 일방, 혼인, 부부, 타인, 번복, 소유
토픽 1: 결정, 종합_고려, 영향, 기간, 고려, 혼인, 기준, 생활
토픽 2: 민법, 불법행위, 정신_고통, 피해자, 제750조에, 손해배상_청구, 입다, 피해
토픽 3: 입증, 피해자, 손해배상_청구, 증거, 손해, 상대방, 또한, 정신
토픽 4: 재산_분할, 민법, 이루어지다, 청구, 합, 규정, 형성, 채권자
토픽 5: 정신_고통, 제3자가, 방해, 침해, 부부, 혼인_본질, 해당, 생활_유지
토픽 6: 정조_의무, 부부, 간통_포함, 행위, 포함, 평, 모든, 이르다
토픽 7: 혼인, 사실혼_관계, 관계, 민법, 의사, 당사자, 판결, 불구
토픽 8: 위자료, 부부, 종합, 피해자, 혼인_기간, 고려,

In [60]:
for num_topics in top_3_topics:
    print(f"\n=== {num_topics}개 토픽 결과 ===")
    model = models_dict[num_topics]
    
    for topic_id in range(num_topics):
        terms = model.show_topic(topic_id, topn=8)
        topic_str = ', '.join([f"{word}({weight:.4f})" for word, weight in terms])
        print(f"토픽 {topic_id}: {topic_str}")


=== 7개 토픽 결과 ===
토픽 0: 혼인(0.0547), 부부(0.0336), 실질(0.0325), 생활(0.0256), 파탄(0.0237), 입증(0.0208), 상태(0.0182), 관계(0.0178)
토픽 1: 결정(0.0364), 영향(0.0317), 피해자(0.0306), 종합_고려(0.0285), 고려(0.0276), 생활(0.0250), 정신_고통(0.0216), 부부(0.0216)
토픽 2: 불법행위(0.0479), 정신_고통(0.0389), 민법(0.0320), 입증(0.0281), 피해자(0.0258), 부부(0.0257), 입다(0.0234), 손해배상_청구(0.0221)
토픽 3: 증거(0.0348), 입증(0.0339), 법원(0.0232), 소송(0.0217), 이르다(0.0197), 법적(0.0188), 행위(0.0182), 상대방(0.0181)
토픽 4: 재산_분할(0.0362), 민법(0.0292), 청구(0.0199), 이루어지다(0.0151), 기여(0.0139), 양육비(0.0127), 기준(0.0122), 포함(0.0113)
토픽 5: 정신_고통(0.0506), 방해(0.0447), 제3자가(0.0423), 혼인_본질(0.0413), 해당(0.0370), 생활_유지(0.0326), 침해(0.0319), 부부(0.0315)
토픽 6: 정조_의무(0.0641), 부부(0.0422), 간통_포함(0.0367), 행위(0.0325), 평(0.0302), 포함(0.0292), 모든(0.0257), 제3자가(0.0246)

=== 9개 토픽 결과 ===
토픽 0: 참작_평(0.0450), 파탄_초래(0.0426), 일방(0.0344), 혼인(0.0281), 부부(0.0272), 타인(0.0240), 번복(0.0215), 소유(0.0215)
토픽 1: 결정(0.0400), 종합_고려(0.0364), 영향(0.0361), 기간(0.0296), 고려(0.0295), 혼인(0.0290), 기준(0.0286), 생활(0.0281)
토픽

# 4단계: 최종 토픽 라벨 할당

In [77]:
# 8개 토픽 모델 선택
optimal_num_topics = 9
best_model = models_dict[optimal_num_topics]  # coherence 결과에서 9개 토픽 모델

print(f"선택된 토픽 수: {optimal_num_topics}")

선택된 토픽 수: 9


In [96]:
df_clean2 = df_clean.copy()

In [97]:
# 각 문서가 어떤 토픽에 속하는지 결정하는 단계
def assign_document_topics(model, corpus):
    topic_assignments = []
    topic_probabilities = []
    
    for doc_bow in corpus:
        doc_topics = model.get_document_topics(doc_bow)
        
        if doc_topics:
            # 가장 높은 확률의 토픽 찾기
            dominant_topic = max(doc_topics, key=lambda x: x[1])
            topic_assignments.append(dominant_topic[0])      # 토픽 번호
            topic_probabilities.append(dominant_topic[1])    # 확률
        else:
            topic_assignments.append(-1)
            topic_probabilities.append(0.0)
    
    return topic_assignments, topic_probabilities

# 실행
topic_ids, topic_probs = assign_document_topics(best_model, corpus)

# DataFrame에 추가
df_clean2['topic_label'] = topic_ids
df_clean2['topic_probability'] = topic_probs

In [ ]:
=== 9개 토픽 결과 ===
토픽 0: 참작_평(0.0450), 파탄_초래(0.0426), 일방(0.0344), 혼인(0.0281), 부부(0.0272), 타인(0.0240), 번복(0.0215), 소유(0.0215)
토픽 1: 결정(0.0400), 종합_고려(0.0364), 영향(0.0361), 기간(0.0296), 고려(0.0295), 혼인(0.0290), 기준(0.0286), 생활(0.0281)
토픽 2: 민법(0.0467), 불법행위(0.0461), 정신_고통(0.0330), 피해자(0.0328), 제750조에(0.0260), 손해배상_청구(0.0258), 입다(0.0253), 피해(0.0223)
토픽 3: 입증(0.0656), 피해자(0.0373), 손해배상_청구(0.0324), 증거(0.0309), 손해(0.0301), 상대방(0.0277), 또한(0.0255), 정신(0.0239)
토픽 4: 재산_분할(0.0432), 민법(0.0303), 이루어지다(0.0224), 청구(0.0186), 합(0.0154), 규정(0.0134), 형성(0.0132), 채권자(0.0131)
토픽 5: 정신_고통(0.0509), 제3자가(0.0431), 방해(0.0365), 침해(0.0357), 부부(0.0357), 혼인_본질(0.0354), 해당(0.0337), 생활_유지(0.0269)
토픽 6: 정조_의무(0.0699), 부부(0.0413), 간통_포함(0.0400), 행위(0.0337), 포함(0.0325), 평(0.0324), 모든(0.0272), 이르다(0.0256)
토픽 7: 혼인(0.0270), 사실혼_관계(0.0250), 관계(0.0247), 민법(0.0225), 의사(0.0209), 당사자(0.0207), 판결(0.0190), 불구(0.0187)
토픽 8: 위자료(0.0240), 부부(0.0237), 종합(0.0198), 피해자(0.0197), 혼인_기간(0.0193), 고려(0.0191), 결정(0.0190), 혼인(0.0179)


In [109]:
topic_labels = {
    0: "혼인 파탄의 귀책사유와 법적 평가",      
    1: "법원의 판결 결정 시 종합적 고려사항",     
    2: "민법상 불법행위와 정신적 손해배상 (제750조)",       
    3: "소송에서의 손해 및 피해 사실 입증 책임",   
    4: "민법상 재산분할 원칙과 채무 문제",     
    5: "제3자의 혼인 본질 침해와 정신적 피해",     
    6: "부부의 정조 의무와 부정행위의 정의",    
    7: "사실혼 관계의 성립 요건 및 법적 효력",
    8: "위자료 액수 산정 시 고려 요소"
    
}

# DataFrame에 토픽 이름 추가
df_clean2['topic_name'] = df_clean2['topic_label'].map(topic_labels)

In [99]:
# 1. 토픽별 대표 문서 확인
def show_topic_examples(df, topic_label, n_examples=3):
    topic_docs = df[df['topic_label'] == topic_id].sort_values('topic_probability', ascending=False)
    
    print(f"\n=== 토픽 {topic_id} 대표 문서 ===")
    for i, (idx, row) in enumerate(topic_docs.head(n_examples).iterrows()):
        print(f"{i+1}. (확률: {row['topic_probability']:.3f})")
        print(f"   {row['input'][:100]}...")

# 모든 토픽 확인
for topic_id in range(9):
    show_topic_examples(df_clean2, topic_id, n_examples=2)

# 2. 확률 분포 확인
print("토픽 확률 통계:")
print(df_clean2['topic_probability'].describe())

# 3. 애매한 할당 확인 (확률 < 0.3)
low_confidence = df_clean2[df_clean2['topic_probability'] < 0.3]
print(f"애매한 할당: {len(low_confidence)}개 ({len(low_confidence)/len(df_clean2)*100:.1f}%)")



=== 토픽 0 대표 문서 ===
1. (확률: 0.982)
   부부 간 임대차계약에서 임대차보증금 반환채권의 소유 여부를 입증하기 위한 조건은 무엇인가요?...
2. (확률: 0.972)
   부부일방이 혼인 중에 자기 명의로 취득한 재산에 대해 어떤 법리가 적용되나요?...

=== 토픽 1 대표 문서 ===
1. (확률: 0.981)
   부부의 일방이 제3자를 상대로 한 손해배상청구에서 위자료의 액수는 어떤 기준으로 결정되나요?...
2. (확률: 0.980)
   정신적 고통에 대한 손해배상의 범위는 어떻게 정해지나요?...

=== 토픽 2 대표 문서 ===
1. (확률: 0.979)
   부부는 혼인 후 어떤 법적 의무를 가지나요?...
2. (확률: 0.976)
   부부가 서로에게 부담하는 의무는 무엇인가요?...

=== 토픽 3 대표 문서 ===
1. (확률: 0.981)
   부정행위로 인한 정신적 손해배상청구 소송에서 피해자가 지켜야 할 절차는 무엇인가요?...
2. (확률: 0.981)
   정신적 손해를 입은 피해자가 위자료를 청구하기 위해 입증해야 할 사항은 무엇인가요?...

=== 토픽 4 대표 문서 ===
1. (확률: 0.983)
   양육비의 변경을 청구하기 위해 법원에 제출해야 하는 증거의 종류는 무엇인가요?...
2. (확률: 0.982)
   재산분할 청구에서 법원이 재산분할의 방법이나 비율을 결정할 때 고려해야 하는 주요 요소는 무엇인가요?...

=== 토픽 5 대표 문서 ===
1. (확률: 0.982)
   부부 중 한 사람이 제3자와 부정행위를 할 경우, 제3자가 직면할 수 있는 법적 책임은 무엇인가요?...
2. (확률: 0.981)
   부정행위로 인해 제3자가 부부의 혼인관계를 방해하면 법적인 책임이 있나요?...

=== 토픽 6 대표 문서 ===
1. (확률: 0.975)
   부부의 일방과 제3자가 부정행위를 저질렀을 때, 배우자가 제3자를 상대로 손해배상을 청구할 수 있나요?...
2. (확률

In [110]:
# 이 2개는 어떤 문서인지 확인해보면 좋을 것 같아요
low_confidence = df_clean2[df_clean2['topic_probability'] < 0.3]
print("애매한 할당 문서들:")
for idx, row in low_confidence.iterrows():
    print(f"- {row['input'][:100]}...")

애매한 할당 문서들:
- 불법행위에 기한 손해배상에서 위자료의 산정기준은 어떻게 되나요?...
- 사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위한 기준은 무엇인가요?...


In [106]:
# 1. 새로운 토픽 ID를 변수에 저장합니다.
new_topic_id = 7

# 2. 628번 행의 'dominant_topic' (숫자 ID)를 먼저 변경합니다.
df_clean2.loc[628, 'topic_label'] = new_topic_id


In [104]:
row_data = df_clean2.loc[628]
print(row_data)


doc_id                                          의정부지방법원고양지원-2018가단79069
casetype                                                          civil
casenames                                                           대여금
input                      사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위한 기준은 무엇인가요?
output                사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위해서는 당사자 간에 사실혼...
fileName                                        민사법_판결문_질의응답_17592.json
announce_date                             2019-07-11T09:00:00.000+09:00
is_divorce                                                            1
text_combined         사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위한 기준은 무엇인가요? 사...
input_processed                                사실혼 관계 해소 원인 재산 분할 청구 기준
topic_label                                       사실혼 관계의 성립 요건 및 법적 효력
combined_processed    사실혼 관계 해소 원인 재산 분할 청구 기준 당사자 실제 존재 입증 존부 가족 체감...
ngram_text            사실혼_관계 해소 원인 재산_분할 청구 기준 당사자 실제 존재 입증 존부 가족 체감...
topic_probability                                              0

In [111]:
low_confidence

,doc_id,casetype,casenames,input,output,fileName,announce_date,is_divorce,text_combined,input_processed,topic_label,combined_processed,ngram_text,topic_probability,topic_name
133,서울서부지방법원-2016가단252297,civil,손해배상(기),불법행위에 기한 손해배상에서 위자료의 산정기준은 어떻게 되나요?,"불법행위에 기한 손해배상의 경우, 위자료의 액수는 피해자가 입은 정신적 고통의 정도...",민사법_판결문_질의응답_51369.json,2018-01-19T09:00:00.000+09:00,1,불법행위에 기한 손해배상에서 위자료의 산정기준은 어떻게 되나요? 불법행위에 기한 손...,불법행위 기한 손해배상 위자료 산정 기준,8,불법행위 기한 손해배상 위자료 산정 기준 액수 피해자 입다 정신 고통 가해자 행위 ...,불법행위 기한 손해배상 위자료 산정_기준 액수 피해자 입다 정신_고통 가해자 행위 ...,0.290355,위자료 액수 산정 시 고려 요소
628,의정부지방법원고양지원-2018가단79069,civil,대여금,사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위한 기준은 무엇인가요?,사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위해서는 당사자 간에 사실혼...,민사법_판결문_질의응답_17592.json,2019-07-11T09:00:00.000+09:00,1,사실혼 관계 해소를 원인으로 한 재산분할청구가 인정되기 위한 기준은 무엇인가요? 사...,사실혼 관계 해소 원인 재산 분할 청구 기준,7,사실혼 관계 해소 원인 재산 분할 청구 기준 당사자 실제 존재 입증 존부 가족 체감...,사실혼_관계 해소 원인 재산_분할 청구 기준 당사자 실제 존재 입증 존부 가족 체감...,0.291537,사실혼 관계의 성립 요건 및 법적 효력


In [112]:
# 모든 토픽 이름 출력해서 확인
print("=== 전체 토픽 이름 목록 ===")
all_topics = df_clean2['topic_name'].value_counts()
for topic_name, count in all_topics.items():
    print(f"{topic_name}: {count}개")

=== 전체 토픽 이름 목록 ===
제3자의 혼인 본질 침해와 정신적 피해: 215개
민법상 재산분할 원칙과 채무 문제: 92개
법원의 판결 결정 시 종합적 고려사항: 87개
부부의 정조 의무와 부정행위의 정의: 77개
위자료 액수 산정 시 고려 요소: 72개
민법상 불법행위와 정신적 손해배상 (제750조): 71개
소송에서의 손해 및 피해 사실 입증 책임: 47개
사실혼 관계의 성립 요건 및 법적 효력: 37개
혼인 파탄의 귀책사유와 법적 평가: 13개


In [129]:
df = pd.read_csv("C:/Users/82105/Downloads/divorce_topic_labeled_.csv", encoding="utf-8-sig")

In [134]:
# 상위 카테고리별 포함 토픽 이름 리스트
category_map = {
    "위자료·정신적 손해배상": [
        "위자료 액수 산정 시 고려 요소",'민법상 재산분할 원칙과 채무 문제'
    ],
    "사실혼·법적 효력" : ["사실혼 관계의 성립 요건 및 법적 효력"],
    "소송 절차·입증 책임" : ["소송에서의 손해 및 피해 사실 입증 책임","법원의 판결 결정 시 종합적 고려사항"],
    "유책사유·배상 책임" : [ '혼인 파탄의 귀책사유와 법적 평가','부부의 정조 의무와 부정행위의 정의' ],
    "불법행위_책임": ["제3자의 혼인 본질 침해와 정신적 피해",
                  "민법상 불법행위와 정신적 손해배상 (제750조)"]
}


In [135]:
# topic_name → 상위 카테고리 매핑 함수
def map_to_category(topic_name):
    for category, names in category_map.items():
        if topic_name in names:
            return category
    return "기타"

# 새로운 컬럼 생성
df_clean2['super_category'] = df_clean2['topic_name'].apply(map_to_category)

In [136]:
df_clean2.head()

,doc_id,casetype,casenames,input,output,fileName,announce_date,is_divorce,text_combined,input_processed,topic_label,combined_processed,ngram_text,topic_probability,topic_name,super_category
0,대전지방법원-2016나2411,civil,대여금,혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?,혼인 중 발생한 금전거래에 대한 차용금 반환 청구권이 인정되기 위해서는 차용금 지급...,민사법_판결문_질의응답_10079.json,2016-08-19T09:00:00.000+09:00,1,혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요...,혼인 금전 거래 차용 반환 청구권,4,혼인 금전 거래 차용 반환 청구권 지급 입증 객관 금융 자료 또한 변제 독촉 증거 ...,혼인 금전 거래 차용 반환 청구권 지급 입증 객관 금융 자료 또한 변제 독촉 증거 ...,0.385110,민법상 재산분할 원칙과 채무 문제,위자료·정신적 손해배상
1,대전지방법원-2018나108412,civil,손해배상(기),제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?,제3자는 타인의 부부공동생활에 개입하여 그 파탄을 초래하거나 본질적인 부부공동생활을...,민사법_판결문_질의응답_10088.json,2019-04-02T00:00:00.000+09:00,1,제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요? 제3자는...,부부 일방 부정행위 법적 책임 지게 제3자가,5,부부 일방 부정행위 법적 책임 지게 타인 생활 개입 파탄 초래 본질 방해 서다 침해...,부부_일방 부정행위 법적_책임 지게 타인 생활_개입 파탄_초래 본질 방해 서다 침해...,0.841921,제3자의 혼인 본질 침해와 정신적 피해,불법행위_책임
2,대전지방법원-2022가단102892,civil,손해배상(기),민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?,민법 제840조 제1호에 따른 재판상 이혼사유인 부정한 행위는 간통에 이르지 아니하...,민사법_판결문_질의응답_10114.json,2022-11-16T09:00:00.000+09:00,1,민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가...,민법 규정 재판 사유 부정 행위 의미 제840조 제1호에서,6,민법 규정 재판 사유 부정 행위 의미 간통 이르다 부부 정조 의무 일체 부정행위 포...,민법 규정 재판 사유 부정_행위 의미 간통 이르다 부부 정조_의무 일체 부정행위 포...,0.636486,부부의 정조 의무와 부정행위의 정의,유책사유·배상 책임
3,대전지방법원천안지원-2018가단105206,civil,손해배상(기),배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?,"위자료의 액수는 부정행위의 내용, 정도 및 기간, 결혼기간 및 가족관계, 부정행위가...",민사법_판결문_질의응답_10149.json,2018-10-31T09:00:00.000+09:00,1,배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요? 위자료...,배우자 부정행위 위자료 액수 정해지다,1,배우자 부정행위 위자료 액수 정해지다 기간 결혼 가족 관계 부부 생활 영향 참작 장...,배우자 부정행위 위자료_액수 정해지다 기간 결혼 가족_관계 부부 생활 영향 참작 장...,0.971876,법원의 판결 결정 시 종합적 고려사항,소송 절차·입증 책임
4,서울중앙지방법원-2017가단5177638,civil,구상금,사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...,"사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 경우는, ...",민사법_판결문_질의응답_10839.json,2018-05-11T09:00:00.000+09:00,1,사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...,사해행위 취소 원상회복 청구 소송 다른 채권자 부적 법해,4,사해행위 취소 원상회복 청구 소송 채권자 부적 법해 대해 진행 승소 판결 확정 재산...,사해행위_취소 원상회복 청구_소송 채권자 부적 법해 대해 진행 승소 판결 확정 재산...,0.977511,민법상 재산분할 원칙과 채무 문제,위자료·정신적 손해배상


In [ ]:
# 위자료_관련 문서 수
alimony_broad = df_clean2[df_clean2['super_category'] == "위자료_관련"]
print(f"폭넓은 위자료 관련 사건: {len(alimony_broad)}개")

# 토픽별 건수
print(alimony_broad['topic_name'].value_counts())

In [143]:
df_clean2

,doc_id,casetype,casenames,input,output,fileName,announce_date,is_divorce,text_combined,input_processed,topic_label,combined_processed,ngram_text,topic_probability,topic_name,super_category
0,대전지방법원-2016나2411,civil,대여금,혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요?,혼인 중 발생한 금전거래에 대한 차용금 반환 청구권이 인정되기 위해서는 차용금 지급...,민사법_판결문_질의응답_10079.json,2016-08-19T09:00:00.000+09:00,1,혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요...,혼인 금전 거래 차용 반환 청구권,4,혼인 금전 거래 차용 반환 청구권 지급 입증 객관 금융 자료 또한 변제 독촉 증거 ...,혼인 금전 거래 차용 반환 청구권 지급 입증 객관 금융 자료 또한 변제 독촉 증거 ...,0.385110,민법상 재산분할 원칙과 채무 문제,위자료·정신적 손해배상
1,대전지방법원-2018나108412,civil,손해배상(기),제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요?,제3자는 타인의 부부공동생활에 개입하여 그 파탄을 초래하거나 본질적인 부부공동생활을...,민사법_판결문_질의응답_10088.json,2019-04-02T00:00:00.000+09:00,1,제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요? 제3자는...,부부 일방 부정행위 법적 책임 지게 제3자가,5,부부 일방 부정행위 법적 책임 지게 타인 생활 개입 파탄 초래 본질 방해 서다 침해...,부부_일방 부정행위 법적_책임 지게 타인 생활_개입 파탄_초래 본질 방해 서다 침해...,0.841921,제3자의 혼인 본질 침해와 정신적 피해,불법행위_책임
2,대전지방법원-2022가단102892,civil,손해배상(기),민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요?,민법 제840조 제1호에 따른 재판상 이혼사유인 부정한 행위는 간통에 이르지 아니하...,민사법_판결문_질의응답_10114.json,2022-11-16T09:00:00.000+09:00,1,민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가...,민법 규정 재판 사유 부정 행위 의미 제840조 제1호에서,6,민법 규정 재판 사유 부정 행위 의미 간통 이르다 부부 정조 의무 일체 부정행위 포...,민법 규정 재판 사유 부정_행위 의미 간통 이르다 부부 정조_의무 일체 부정행위 포...,0.636486,부부의 정조 의무와 부정행위의 정의,유책사유·배상 책임
3,대전지방법원천안지원-2018가단105206,civil,손해배상(기),배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요?,"위자료의 액수는 부정행위의 내용, 정도 및 기간, 결혼기간 및 가족관계, 부정행위가...",민사법_판결문_질의응답_10149.json,2018-10-31T09:00:00.000+09:00,1,배우자가 있는 사람과 부정행위를 한 경우 위자료의 액수는 어떻게 정해지나요? 위자료...,배우자 부정행위 위자료 액수 정해지다,1,배우자 부정행위 위자료 액수 정해지다 기간 결혼 가족 관계 부부 생활 영향 참작 장...,배우자 부정행위 위자료_액수 정해지다 기간 결혼 가족_관계 부부 생활 영향 참작 장...,0.971876,법원의 판결 결정 시 종합적 고려사항,소송 절차·입증 책임
4,서울중앙지방법원-2017가단5177638,civil,구상금,사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...,"사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 경우는, ...",민사법_판결문_질의응답_10839.json,2018-05-11T09:00:00.000+09:00,1,사해행위 취소 및 원상회복청구 소송에서 다른 채권자의 청구가 부적법해지는 상황은 어...,사해행위 취소 원상회복 청구 소송 다른 채권자 부적 법해,4,사해행위 취소 원상회복 청구 소송 채권자 부적 법해 대해 진행 승소 판결 확정 재산...,사해행위_취소 원상회복 청구_소송 채권자 부적 법해 대해 진행 승소 판결 확정 재산...,0.977511,민법상 재산분할 원칙과 채무 문제,위자료·정신적 손해배상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,서울중앙지방법원-2019가단5097163,civil,"성공보수금청구의소,부당이득금",이혼 및 재산분할 청구 사건에서의 조정 절차의 법률적 의미와 절차는 무엇인가요?,이혼 및 재산분할 청구 사건에서의 조정 절차는 법원이 쌍방의 입장을 고려하여 원만한...,민사법_판결문_질의응답_84137.json,2020-02-20T09:00:00.000+09:00,1,이혼 및 재산분할 청구 사건에서의 조정 절차의 법률적 의미와 절차는 무엇인가요? 이...,재산 분할 청구 사건 조정 법률 의미,4,재산 분할 청구 사건 조정 법률 의미 법원 쌍방 입장 고려 해결 유도 제도로 당사자...,재산_분할 청구 사건 조정 법률 의미 법원 쌍방 입장 고려 해결 유도 제도로 당사자...,0.711038,민법상 재산분할 원칙과 채무 문제,위자료·정신적 손해배상
730,서울중앙지방법원-2015가단206346,civil,대여금등,이혼 시 재산분할의 법적 근거 및 조건은 무엇인가요?,"민법 제839조의2에 따르면, 이혼 시 재산분할은 부부가 혼인 중에 형성한 공동재산...",민사법_판결문_질의응답_84748.json,2017-05-30T09:00:00.000+09:00,1,"이혼 시 재산분할의 법적 근거 및 조건은 무엇인가요? 민법 제839조의2에 따르면,...",재산 분할 법적,8,재산 분할 법적 민법 조 부부 혼인 형성 정산 분배 당사자 기여 도르다 고려 유책 ...,재산_분할 법적 민법_조 부부 혼인 형성 정산 분배 당사자 기여 도르다 고려 유책 ...,0.792790,위자료 액수 산정 시 고려 요소,위자료·정신적 손해배상
731,수원지방법원평택지원-2017고합60,criminal,"강간치상,강간,특수재물손괴,자동차손해배상보장법위반,명예훼손","협의이혼절차 중 강간 행위가 발생한 경우, 법적 처벌은 어떻게 이루어지나요?","형법 제297조에 따르면, 사람의 의사에 반하여 강제로 성관계를 가지는 경우 강간죄...",민사법_판결문_질의응답_8700.json,2017-11-06T09:00:00.000+09:00,1,"협의이혼절차 중 강간 행위가 발생한 경우, 법적 처벌은 어떻게 이루어지나요? 형법 ...",협의 강간 행위 법적 처벌 이루어지다,7,협의 강간 행위 법적 처벌 이루어지다 형법 의사 강제 성관계 강간죄 이상 유기 징역...,협의 강간 행위 법적 처벌 이루어지다 형법 의사 강제 성관계 강간죄 이상 유기 징역...,0.592408,사실혼 관계의 성립 요건 및 법적 효력,사실혼·법적 효력
732,부산가정법원-2020느단200572,household,"(본심판)친권자및양육자의변경,(반심판)양육비",면접교섭의 조건과 절차는 어떻게 정해지나요?,면접교섭의 조건과 절차는 사건본인의 복리와 정서적 안정을 최우선으로 고려하여 정해져...,민사법_판결문_질의응답_89243.json,2020-11-26T09:00:00.000+09:00,1,면접교섭의 조건과 절차는 어떻게 정해지나요? 면접교섭의 조건과 절차는 사건본인의 복...,면접 교섭 정해지다,1,면접 교섭 정해지다 사건 본인 복리 정서 안정 우선 고려 청구인 심판 상대방 협의 ...,면접 교섭 정해지다 사건 본인 복리 정서 안정 우선 고려 청구인 심판 상대방 협의 ...,0.602166,법원의 판결 결정 시 종합적 고려사항,소송 절차·입증 책임


In [144]:
# 최종 결과 저장
final_columns = ['input', 'output', 
       'announce_date', 'is_divorce', 'text_combined', 
       'topic_label', 'combined_processed', 'ngram_text', 'topic_probability',
       'topic_name', 'super_category']

final_df = df_clean2[final_columns].copy()
final_df.to_csv("C:/Users/82105/Downloads/divorce_topic_labeled_combined.csv", index=False, encoding='utf-8-sig')
print("최종 결과 저장 완료!")

최종 결과 저장 완료!
